# White House visitor logs

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [19]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import glob
import datetime as dt

### Read the page to find links to raw data

In [6]:
r = requests.get("https://www.whitehouse.gov/disclosures/visitor-logs/")
soup = BeautifulSoup(r.text, "html.parser")

In [7]:
links = [a.get("href") for a in soup.find_all("a", href=re.compile(".csv"))]

In [8]:
for link in links:
    !wget -P 'data/raw/csv' {link} --quiet -N

In [9]:
path = "data/raw/csv/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

src = pd.concat(li, axis=0, ignore_index=True)

### Clean up the columns

In [10]:
src.columns = src.columns.str.lower()

In [11]:
src = src[src.columns.drop(list(src.filter(regex="unnamed")))]

In [12]:
src = src.sort_values("releasedate", ascending=False)

In [13]:
src.head()

,namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,...,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,releasedate,description
21913,FINKELSTEIN,MORGAN,A,U25158,NaN,VA,NaN,NaN,NaN,NaN,...,MB,Browne,Martin,OEOB,97,BROWNE,MARTIN,NaN,8/30/2021,NaN
21836,DELKER,BRIAN,N,U25032,177630.0,VA,5/19/2021 9:27,K0101,5/19/2021 11:07,K01,...,GH,Holt,Charles,NEOB,SB235,HALL,GREGORY,NaN,8/30/2021,NaN
21822,SMITH,SARAH,E,U24917,177882.0,VA,5/18/2021 13:49,B0401,NaN,NaN,...,LM,Mazzeo,Lou,OEOB,330,MAZZEO,LOUIS,NaN,8/30/2021,NaN
21823,VENEZIA,ERICA,L,U24917,177993.0,VA,5/18/2021 13:50,B0401,5/18/2021 16:21,D03,...,LM,Mazzeo,Lou,OEOB,330,MAZZEO,LOUIS,NaN,8/30/2021,NaN
21824,MARCHICK,DAVID,M,U24932,178020.0,VA,5/18/2021 15:15,K0102,5/18/2021 18:17,K01,...,PS,Sullivan,Patrick,NEOB,10th Floor,SULLIVAN,PATRICK,NaN,8/30/2021,NaN


### Real dates

In [14]:
date_cols = [
    "appt_made_date",
    "appt_start_date",
    "appt_end_date",
    "appt_cancel_date",
    "releasedate",
]

In [15]:
src[date_cols] = src[date_cols].applymap(lambda x: pd.to_datetime(x))

### The dataframe has inconsitent casing. Let's make it all upper for grouping later. 

In [16]:
src[
    [
        "namelast",
        "namefirst",
        "namemid",
        "uin",
        "access_type",
        "toa",
        "poa",
        "tod",
        "pod",
        "last_updatedby",
        "post",
        "lastentrydate",
        "terminal_suffix",
        "visitee_namelast",
        "visitee_namefirst",
        "meeting_loc",
        "meeting_room",
        "caller_name_last",
        "caller_name_first",
        "description",
    ]
] = src[
    [
        "namelast",
        "namefirst",
        "namemid",
        "uin",
        "access_type",
        "toa",
        "poa",
        "tod",
        "pod",
        "last_updatedby",
        "post",
        "lastentrydate",
        "terminal_suffix",
        "visitee_namelast",
        "visitee_namefirst",
        "meeting_loc",
        "meeting_room",
        "caller_name_last",
        "caller_name_first",
        "description",
    ]
].apply(
    lambda x: x.astype(str).str.upper()
)

In [17]:
df = src.copy()

---

## Exports

In [20]:
today = dt.date.today().strftime("%m-%d-%Y")

In [21]:
df.to_csv("data/processed/log_" + today + ".csv", index=False)
df.to_csv("data/processed/log_latest.csv", index=False)
src.to_csv("data/raw/log_" + today + ".csv", index=False)